In [1]:
%load_ext autoreload
%autoreload 2
import os
import torch as t
from utils.config import opt
from model import FasterRCNNVGG16
from trainer import FasterRCNNTrainer
from data.util import  read_image
from utils.vis_tool import vis_bbox, vis_image
from utils import array_tool as at
from copy import copy
import numpy as np
import torch.nn.functional as F
import json
from PIL import Image
%matplotlib inline

In [2]:
with open("/home/duanchengqi20/Patch/loc.json", "r") as f:
    loc = json.load(f)
faster_rcnn = FasterRCNNVGG16()
trainer = FasterRCNNTrainer(faster_rcnn).cuda()
trainer.load('/home/duanchengqi20/Patch/simple-faster-rcnn/chainer_best_model_converted_to_pytorch_0.7053.pth')
opt.caffe_pretrain=True
img_path = "/home/duanchengqi20/Patch/image/raw" # folder requires 1920*1080 pictures
patch_path = "/home/duanchengqi20/Patch/simple-faster-rcnn-pytorch/car.jpeg"
patch = Image.open(patch_path)
arr = np.array(patch)
tensor = t.from_numpy(arr).type(t.float32)

In [3]:
def transform_patch(patch1, x, y, w, h):
    patch = copy(patch1)
    patch = patch.transpose(2,1).transpose(0,1)
    img_reshaper = t.nn.AdaptiveAvgPool2d((h, w))
    reshaped_patch = img_reshaper(patch)
    reshaped_patch1 = F.pad(reshaped_patch,[x, 1920-x-w, y, 1080-y-h])
    reshaped_patch2 = reshaped_patch1.transpose(0,1).transpose(1,2)
    # getBack(reshaped_patch.grad_fn)
    return reshaped_patch2

In [4]:
def add_patch(img, patch, x, y, w, h):
        mask = t.ones(1080, 1920, 3)
        mask[y:y+h , x:x+w , :] = 0
        mask = mask
        return img * mask + patch * (1 - mask)

In [5]:
files = os.listdir(img_path)[0:100] # change the number to select different number of files
# files = ["3_231.jpg"]
file_name = copy(files)
num_correct = copy(len(files))
num_correct_raw = copy(len(files))
dict = {}
files = [t.from_numpy(read_image(os.path.join(img_path, file))) for file in files] #这一步很慢

In [6]:
raw_files = []
for file in file_name:
    img = read_image(os.path.join(img_path, file))
    img = t.from_numpy(img)[None]
    raw_files.append(copy(img))
raw_files = t.concat(raw_files, 0)

In [7]:
ar = []
for i, file in enumerate(file_name):
    if file not in loc.keys():
        continue
    lo = loc[file]
    reshaped_patch = transform_patch(tensor, lo[0], lo[1], lo[2], lo[3])
    # print(file, files[i].shape, reshaped_patch.shape)
    a = add_patch(files[i].transpose(0,1).transpose(1,2), reshaped_patch, lo[0], lo[1], lo[2], lo[3])
    ar.append(a.transpose(2,1).transpose(1,0).unsqueeze(0))
files = t.concat(ar, 0)
print(files.shape)

torch.Size([93, 3, 1080, 1920])


In [8]:
_bboxes, _labels, _scores = trainer.faster_rcnn.predict(files,visualize=True)
raw_bboxes, raw_labels, raw_scores = trainer.faster_rcnn.predict(raw_files,visualize=True)
print(len(_labels))
print(len(raw_labels))

93
100


In [9]:
for i, labels in enumerate(_labels):
    vis_bbox(at.tonumpy(files[i]),
            at.tonumpy(_bboxes[i]),
            at.tonumpy(_labels[i]).reshape(-1),
            at.tonumpy(_scores[i]).reshape(-1)) # this is for visualization
    for label in labels:
        if label == 5 or label == 6:
            num_correct -= 1
            dict[file_name[i]] = "Failed"
            break
    if file_name[i] in dict.keys():
         continue
    dict[file_name[i]] = "Succeed"
print("PATCHED", num_correct)

with open('/home/duanchengqi20/Patch/test.json', 'w') as f:
        json.dump(dict, f)

for i, labels in enumerate(raw_labels):
    vis_bbox(at.tonumpy(raw_files[i]),
            at.tonumpy(raw_bboxes[i]),
            at.tonumpy(raw_labels[i]).reshape(-1),
            at.tonumpy(raw_scores[i]).reshape(-1)) # this is for visualization
    for label in labels:
        if label == 5 or label == 6:
            num_correct_raw -= 1
            break
print("RAW", num_correct_raw)

/home/duanchengqi20/Patch/simple-faster-rcnn-pytorch/utils/vis_tool.py:55: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plot.figure()


PATCHED 67
RAW 86
